In [ ]:
%pylab inline
from scipy.special import j0, j1
import numpy as np
import itertools as itrt
import matplotlib.pyplot as plt
#%config Completer.use_jedi = False

In [ ]:
def gamma_to_beta(gamma):
    return sqrt(1-1/gamma**2)
def kinetic_to_betagamma(energy, rest_energy):
    gamma = energy/rest_energy+1
    beta = gamma_to_beta(gamma)
    return beta, gamma

In [ ]:
pcount = 5
m = 1.007319468  #amu
c = 299.792458 # mm/ns
q = 1.178734E-5 ##1.6E-19 C => 4.8032047E-10 statC [cm^(3/2)*g^(1/2)*s^(-1)] => [mm^(3/2)*amu^(1/2)*ns^(-1)];
aperture = 500 #mm
wall_pos = 3000 #wall location on beam axis 'z', relative to origin


Px = np.random.normal(-3e-2, 3e-2, pcount) #3e-2 amu*mm/ns corresponds to 93 keV
Py = np.random.uniform(-3e-2, 3e-2, pcount)
Pz = np.random.uniform(3.4e2, 3.41e2, pcount)  # approx 1 GeV +- 4 MeV, CAREFUL WE ARE IN LAB FRAME HERE
Pt = np.sqrt( Px**2+Py**2+Pz**2+m**2*c**2)
gamma = Pt/(m*c)
bx = Px/(gamma*m*c)
by = Py/(gamma*m*c)
bz = Pz/(gamma*m*c)
beta_avg  = np.sqrt(bx**2+by**2+bz**2)

x = numpy.random.uniform(-50, 50, pcount)
y = numpy.random.uniform(-50, 50, pcount)
z = numpy.random.uniform(-500, -450, pcount)
t = numpy.random.uniform(0, 100/(beta_avg*c), pcount)

bdotx = bx*np.random.uniform(-1e-3,1e-3)  #just a guess, there is probably much more acceleration going on in the core
bdoty = by*np.random.uniform(-1e-3,1e-3) 
bdotz = bz*np.random.uniform(-1e-7,1e-7) #sloppy approximation to reflect much less deviation in logitudinal direction

In [ ]:
init_beam = {'x':x, 'y':y, 'z':z, 't':t, 'Px':Px, 'Py':Py, 'Pz':Pz,'Pt':Pt,
             'bx':bx,'by':by,'bz':bz,'bdotx':bdotx,'bdoty':bdoty,'bdotz':bdotz,'gamma':gamma,'q':q} 

In [ ]:
init_beam['gamma']

In [ ]:
wallcount = pcount
q_w = -q # one-to-one correlation for flat wall
Px_w = np.zeros(wallcount)
Py_w = np.zeros(wallcount)
Pz_w = np.zeros(wallcount)  
Pt_w = np.zeros(wallcount)
gamma_w = np.zeros(wallcount)
bx_w = np.zeros(wallcount)
by_w = np.zeros(wallcount)
bz_w = np.zeros(wallcount)
beta_avg_w = np.sqrt(bx**2+by**2+bz**2)

x_w = np.zeros(wallcount)
y_w = np.zeros(wallcount)
z_w = np.zeros(wallcount)
t_w = t  #????? we need to initialize to some common timestep, I think this is consistent (we operate completely in the lab frame throughout)

bdotx_w = np.zeros(wallcount)  #just a guess, there is probably much more acceleration going on in the core
bdoty_w = np.zeros(wallcount) 
bdotz_w = np.zeros(wallcount) 

In [ ]:
init_wall = {'x':x_w, 'y':y_w, 'z':z_w, 't':t_w, 'Px':Px_w, 'Py':Py_w, 'Pz':Pz_w,'Pt':Pt_w,
             'bx':bx_w,'by':by_w,'bz':bz_w,'bdotx':bdotx_w,'bdoty':bdoty_w,'bdotz':bdotz_w,'gamma':gamma_w,'q':q_w} 

In [ ]:
def bouncer_flat(vector,vector_ext,wall_Z,apt_R):
    """ 
    taking losses at the circular aperture and generating a full image bunch reflecting off a flat wall
    
    includes cutoffs for particles striking wall or passing through the aperture
    """
    for i in range(len(vector['x'])):
        r = np.sqrt(vector['x'][i]**2+vector['y'][i]**2)
        print(r)
        #sloppy way of taking losses, setting charge to zero
        if vector['z'][i]>=wall_Z and r>=apt_R:
            vector['q'][i]=0 #
            
        
        elif vector['z'][i]<wall_Z and r<=apt_R:
            vector_ext['q']=-vector['q']
            vector_ext['z'][i]=wall_Z + 2*(wall_Z-vector['z'][i])
            vector_ext['x'][i]=vector['x'][i]
            vector_ext['y'][i]=vector['y'][i]
            vector_ext['Px'][i]=vector['Px'][i]
            vector_ext['Py'][i]=vector['Py'][i]
            vector_ext['Pz'][i]=-vector['Pz'][i]
            vector_ext['Pt'][i]=vector['Pt'][i] #right?
            vector_ext['gamma'][i]=vector['gamma'][i] 
            vector_ext['bx'][i]=vector['bx'][i] 
            vector_ext['by'][i]=vector['by'][i]
            vector_ext['bz'][i]=-vector['bz'][i] 
            vector_ext['bdotx'][i]=vector['bdotx'][i] 
            vector_ext['bdoty'][i]=vector['bdoty'][i] 
            vector_ext['bdotz'][i]=vector['bdotz'][i] 
            vector_ext['t'][i]=vector['t'][i]   #do NOT retard here, image charge is made to exist at the moment the original charge is created
            
    return(vector,vector_ext)

In [ ]:
bunch_rl,bunch_img = bouncer_flat(init_beam,init_wall,wall_pos,aperture) 
#external_particle = ?????

In [ ]:
# def chrono_matcher(vector_set,vector_ext_set,index_global_t,index_particle):
#     """ 
#     scans back though external particle histories to find matching time point for sample particle and 
    
#     Times are all in lab-frame. vector_set and vector_ext_set are full trajectories. index_global_t is the current time step
#     ONLY WORKS FOR ONE SAMPLE PARTICLE AT A TIME -- But generates matched times for all particles in the corresponding image bunch 
#     """
#     vector_ext_set_matched = copy(vector_ext_set)
#     if index==0:
#         raise Exception('vector_ext_set must have a history -- cannot begin at t=0')
        
#     #rudimentary scan, should be as close to zero as possible, but just overshooting by a time step should be conservative enough for now.
#     for j in range(len(vector_ext_set[index]['t'])):            
#             k = 1
#             while vector_set[index_global_t]['t'][index_particle]-vector_ext_set[index_global_t-k]['t'][j] > 0.1:
#                 k += 1
#             else:
#                 vector_ext_set_matched[index]['t'][j] = vector_ext_set[index-k]['t'][j]
            
# return(vector_ext_set_matched)        
            

In [ ]:
def dist_euclid(vector,vector_ext,index):
    """
    simple Euclidean distance generator with our standard retarded-potential notation
    
    requires chrono_match-ed trajectories
    """
    result = {}
    result['R'] = np.zeros_like(vector['x'])    
    result['nx'] = np.zeros_like(vector['x'])
    result['ny'] = np.zeros_like(vector['x'])    
    result['nz'] = np.zeros_like(vector['x'])    
    for j in range(len(vector_ext['x'])):
        result['R'][j] = np.sqrt( (vector['x'][index]-vector_ext['x'][j])**2+
                          (vector['y'][index]-vector_ext['y'][j])**2+
                          (vector['z'][index]-vector_ext['z'][j])**2 )
        result['nx'][j] = (vector['x'][index]-vector_ext['x'][j])/result['R'][j]
        result['ny'][j] = (vector['y'][index]-vector_ext['y'][j])/result['R'][j]
        result['nz'][j] = (vector['z'][index]-vector_ext['z'][j])/result['R'][j]
    return(result)

In [ ]:
def step1(h, vector,vector_ext): # nhat includes R and fnhat components, need to generate this per particle pair
    result = vector
    for i in range(len(vector['x'])):   #iterating over all real particles OR all reflection points (these must be done in separate steps)            
        nhat = dist_euclid(vector,vector_ext,i)
        for j in range(len(vector_ext['x'])): #summing all external contributions (reflected particles and/or local particles)                        
            result['x'][i] = vector['x'][i]
            result['y'][i] = vector['y'][i]
            result['z'][i] = vector['z'][i]
            result['t'][i] = vector['t'][i]
            result['Px'][i] = vector['Px'][i] + h/m*vector['q']*vector_ext['q']*vector['gamma'][i]\
                        *1/(1-np.dot((vector_ext['bx'][j],vector_ext['by'][j],vector_ext['bz'][j]),(nhat['nx'][j],nhat['ny'][j],nhat['nz'][j]))**2)\
                        *( 1/nhat['R'][j]**2*vector['bx'][i]*np.dot((vector_ext['bx'][j],vector_ext['by'][j],vector_ext['bz'][j]),(vector['bx'][i],vector['by'][i],vector['bz'][i]))\
                        -nhat['nx'][j]*np.dot((vector_ext['bx'][j],vector_ext['by'][j],vector_ext['bz'][j]),(vector['bx'][i],vector['by'][i],vector['bz'][i]))\
                        /(vector_ext['gamma'][j]**2*nhat['R'][j]**2*(1-np.dot((vector_ext['bx'][j],vector_ext['by'][j],vector_ext['bz'][j]),(nhat['nx'][j],nhat['ny'][j],nhat['nz'][j]))))\
                        -nhat['nx'][j]/(c*nhat['R'][j])*(np.dot((vector['bx'][i],vector['by'][i],vector['bz'][i]),((vector_ext['bdotx'][j],vector_ext['bdoty'][j],vector_ext['bdotz'][j])+vector_ext['gamma'][j]**2\
                        *np.dot((vector_ext['bx'][j],vector_ext['by'][j],vector_ext['bz'][j]),np.dot((vector_ext['bx'][j],vector_ext['by'][j],vector_ext['bz'][j]),(vector_ext['bdotx'][j],vector_ext['bdoty'][j],vector_ext['bdotz'][j])) ) ) ) ) )
            result['Py'][i] = vector['Py'][i] + h/m*vector['q']*vector_ext['q']*vector['gamma'][i]\
                        *1/(1-np.dot((vector_ext['bx'][j],vector_ext['by'][j],vector_ext['bz'][j]),(nhat['nx'][j],nhat['ny'][j],nhat['nz'][j]))**2)\
                        *( 1/nhat['R'][j]**2*vector['by'][i]*np.dot((vector_ext['bx'][j],vector_ext['by'][j],vector_ext['bz'][j]),(vector['bx'][i],vector['by'][i],vector['bz'][i]))\
                        -nhat['ny'][j]*np.dot((vector_ext['bx'][j],vector_ext['by'][j],vector_ext['bz'][j]),(vector['bx'][i],vector['by'][i],vector['bz'][i]))\
                        /(vector_ext['gamma'][j]**2*nhat['R'][j]**2*(1-np.dot((vector_ext['bx'][j],vector_ext['by'][j],vector_ext['bz'][j]),(nhat['nx'][j],nhat['ny'][j],nhat['nz'][j]))))\
                        -nhat['ny'][j]/(c*nhat['R'][j])*(np.dot((vector['bx'][i],vector['by'][i],vector['bz'][i]),((vector_ext['bdotx'][j],vector_ext['bdoty'][j],vector_ext['bdotz'][j])+vector_ext['gamma'][j]**2\
                        *np.dot((vector_ext['bx'][j],vector_ext['by'][j],vector_ext['bz'][j]),np.dot((vector_ext['bx'][j],vector_ext['by'][j],vector_ext['bz'][j]),(vector_ext['bdotx'][j],vector_ext['bdoty'][j],vector_ext['bdotz'][j])) ) ) ) ) )
            result['Pz'][i] = vector['Pz'][i] + h/m*vector['q']*vector_ext['q']*vector['gamma'][i]\
                        *1/(1-np.dot((vector_ext['bx'][j],vector_ext['by'][j],vector_ext['bz'][j]),(nhat['nx'][j],nhat['ny'][j],nhat['nz'][j]))**2)\
                        *( 1/nhat['R'][j]**2*vector['bz'][i]*np.dot((vector_ext['bx'][j],vector_ext['by'][j],vector_ext['bz'][j]),(vector['bx'][i],vector['by'][i],vector['bz'][i]))\
                        -nhat['nz'][j]*np.dot((vector_ext['bx'][j],vector_ext['by'][j],vector_ext['bz'][j]),(vector['bx'][i],vector['by'][i],vector['bz'][i]))\
                        /(vector_ext['gamma'][j]**2*nhat['R'][j]**2*(1-np.dot((vector_ext['bx'][j],vector_ext['by'][j],vector_ext['bz'][j]),(nhat['nx'][j],nhat['ny'][j],nhat['nz'][j]))))\
                        -nhat['nz'][j]/(c*nhat['R'][j])*(np.dot((vector['bx'][i],vector['by'][i],vector['bz'][i]),((vector_ext['bdotx'][j],vector_ext['bdoty'][j],vector_ext['bdotz'][j])+vector_ext['gamma'][j]**2\
                        *np.dot((vector_ext['bx'][j],vector_ext['by'][j],vector_ext['bz'][j]),np.dot((vector_ext['bx'][j],vector_ext['by'][j],vector_ext['bz'][j]),(vector_ext['bdotx'][j],vector_ext['bdoty'][j],vector_ext['bdotz'][j])) ) ) ) ) )
            result['Pt'][i] = vector['Pt'][i] + h/m*vector['q']*vector_ext['q']*vector['gamma'][i] \
                        *(((vector_ext['gamma'][j]**2*np.dot((vector_ext['bx'][j],vector_ext['by'][j],vector_ext['bz'][j]),(vector_ext['bdotx'][j],vector_ext['bdoty'][j],vector_ext['bdotz'][j])))/(c*nhat['R'][j]) \
                        - 1/nhat['R'][j]**2) /(1 - np.dot((vector_ext['bx'][j],vector_ext['by'][j],vector_ext['bz'][j]),(nhat['nx'][j],nhat['ny'][j],nhat['nz'][j]))))
            result['gamma'][i] = result['Pt'][i]/(m*c**2)
            result['bx'][i] = result['Px'][i]/(m*c*result['gamma'][i])
            result['by'][i] = result['Py'][i]/(m*c*result['gamma'][i])
            result['bz'][i] = result['Pz'][i]/(m*c*result['gamma'][i])
            result['bdotx'][i] = (result['Px'][i]-vector['Px'][i]) / h   #necessary history to treat as an external particle later (keep in lab frame?) 
            result['bdoty'][i] =(result['Py'][i]-vector['Py'][i]) / h
            result['bdotz'][i] = (result['Pz'][i]-vector['Pz'][i]) / h
            vector = result #will this work? need to update values here between each particle pair calculation
        return result,nhat

        

def step2(h, vector,vector_ext,nhat):
    result = vector
    for i in range(len(vector['x'])):   #iterating over all real particles OR all reflection points (these must be done in separate steps)            
        for j in range(len(vector_ext['x'])): #summing all external contributions (reflected particles and/or local particles) 
            result['x'][i] = vector['x'][i] + h/m * (vector['Px'][i]+vector['q']/c*vector_ext['q']*vector_ext['bx'][j]\
                        / (vector_ext['gamma'][j]*c*nhat['R'][j]*(1-np.dot((vector_ext['bx'][j],vector_ext['by'][j],vector_ext['bz'][j]),(nhat['nx'][j],nhat['ny'][j],nhat['nz'][j])))))
            result['y'][i] = vector['y'][i] + h/m * (vector['Py'][i]+vector['q']/c*vector_ext['q']*vector_ext['by'][j]\
                        / (vector_ext['gamma'][j]*c*nhat['R'][j]*(1-np.dot((vector_ext['bx'][j],vector_ext['by'][j],vector_ext['bz'][j]),(nhat['nx'][j],nhat['ny'][j],nhat['nz'][j])))))
            result['z'][i] = vector['z'][i] + h/m * (vector['Pz'][i]+vector['q']/c*vector_ext['q']*vector_ext['bz'][j]\
                        / (vector_ext['gamma'][j]*c*nhat['R'][j]*(1-np.dot((vector_ext['bx'][j],vector_ext['by'][j],vector_ext['bz'][j]),(nhat['nx'][j],nhat['ny'][j],nhat['nz'][j])))))
            result['t'][i] = vector['t'][i] + h/m * (vector['Pt'][i]+vector['q']/c*vector_ext['q']\
                        / (vector_ext['gamma'][j]*c*nhat['R'][j]*(1-np.dot((vector_ext['bx'][j],vector_ext['by'][j],vector_ext['bz'][j]),(nhat['nx'][j],nhat['ny'][j],nhat['nz'][j])))))
            result['Px'][i] = vector['Px'][i]
            result['Py'][i] = vector['Py'][i]
            result['Pz'][i] = vector['Pz'][i]
            result['Pt'][i] = vector['Pt'][i]
        return result,nhat

In [ ]:
s1out,s1nhat = step1(1E-3,bunch_rl,bunch_img)

In [ ]:
s2out,s2nhat = step2(1E-3,s1out,bunch_img,s1nhat)